In [1]:
import numpy as np
from IPython.display import display
from ipywidgets import interact, interact_manual, widgets,fixed
import gzip

In [7]:
screen_entropy=widgets.Text(value="",description='', disabled=True,layout=widgets.Layout(width='300px'),font_size=12)
def generate_biased_rnd_vector_byte(p_in,n_in):
    v_byte_to_ret=np.zeros((n_in,),dtype=np.uint8)
    v_raw_rnd_bin=np.array(np.random.rand(8*n_in)<p_in,dtype=np.uint8)
    for i in range(8):
        v_byte_to_ret+=(v_raw_rnd_bin[i::8]<<i)
    return v_byte_to_ret
def comp_gzip(v_in):
    v_out = np.frombuffer(gzip.compress(v_in),dtype=np.uint8)
    return v_out
def afficher(v_in,nb_bytes_per_lines=128):
    v_in_len=v_in.shape[0]
    nb_lines=int(v_in_len/nb_bytes_per_lines)+1
   
    for j in range(nb_lines):
        s_line='%.3d:'%(j)
        for i in range(nb_bytes_per_lines):
            k=j*nb_bytes_per_lines+i
            if k<v_in_len:
                s_line='%s%.2x'%(s_line,v_in[k])
        print(s_line)

def gzipVsentropy(p=0.5,nb_bytes=1024):
   
    if p*(1-p)==0:
        entropy=0
    else:
        entropy=(np.log2(p)*(p)+np.log2(1-p)*(1-p))/-1
    screen_entropy.value='p=%.2f correspond a une entropie de %.2f'%(p,entropy)
   
    v_in=generate_biased_rnd_vector_byte(p,nb_bytes)
    v_comp=comp_gzip(v_in)
    print('\nFICHIER NON COMPRESSE : %d BYTES'%(v_in.shape[0]))
    afficher(v_in,104)
    print('\nFICHIER COMPRESSE : %d BYTES'%(v_comp.shape[0]))
    afficher(v_comp,104)

In [23]:
im=interact(gzipVsentropy)
im.widget.close()
im.widget.children[0].max=1.0
im.widget.children[0].min=0.0
im.widget.children[0].step=0.01
im.widget.children[0].description='Probabilite de generer un 1 :'
im.widget.children[0].style={'description_width': '50%'}
im.widget.children[0].layout=widgets.Layout(width='100%')

im.widget.children[1].min=64
im.widget.children[1].max=int(1024*16)
im.widget.children[1].step=8
im.widget.children[1].value=64
im.widget.children[1].description='Nombre de bytes a generer:'
im.widget.children[1].style={'description_width': '50%'}
im.widget.children[1].layout=widgets.Layout(width='100%')

vb0=widgets.VBox([im.widget.children[0],im.widget.children[1],screen_entropy,im.widget.children[2]])

"""
im.widget.children[0].step=0.01
im.widget.children[1].max=4096
im.widget.children[1].min=32
display(im)
"""
display(vb0)

interactive(children=(FloatSlider(value=0.5, description='p', max=1.5, min=-0.5), IntSlider(value=1024, descri…